In [1]:
from __future__ import print_function

import tensorflow as tf
import random

In [2]:
## read text data from file

import csv
import string

labelList = []
tweetList = []
tweetList_test = []

with open('train.csv', encoding='utf-8') as trainfile:
    reader = csv.reader(trainfile)
    for row in reader:
        if row[0] == 'HillaryClinton':
            labelList.append(1)
        elif row[0] == 'realDonaldTrump':
            labelList.append(0)
            
        if row[1] != 'tweet':
            tweetList.append(row[1])
            
with open('test.csv', encoding='utf-8') as testfile:
    reader = csv.reader(testfile)
    for row in reader: 
        if row[1] != 'tweet':
            tweetList_test.append(row[1])

N = len(tweetList)
print('N = %d' % len(tweetList))
print('N = %d' % len(labelList))

N_test = len(tweetList_test)
print('N_test = %d' % len(tweetList_test))

##############################################################

## corpus
import string

def refineWordsList(tweetList):
    ret = []
    for tweet in tweetList:
        step1 = tweet.split('http',1)[0]
        step2 = step1.lower()
        step3 = step2.translate(str.maketrans('.',' '))
        step4 = step3.translate(str.maketrans('','',string.punctuation))
        ret.append(step4)
    return ret

tweetList = refineWordsList(tweetList)
tweetList_test = refineWordsList(tweetList_test)

corpus = set()

for tweet in tweetList:
    words = tweet.split();
    corpus.update(words)

corpus = list(corpus)
M = len(corpus)
print('M = %d' % M)

##############################################################

## Label Encoding

hashEncoder = {}
i = 0
for word in corpus:
    hashEncoder[word] = i
    i += 1

# train
tweetList_encoded = []
for tweet in tweetList:
    tweet_encoded = []
    for word in tweet.split():
        word_encoded = hashEncoder[word]
        tweet_encoded.append(word_encoded)
    tweetList_encoded.append(tweet_encoded)
    
# test
tweetList_test_encoded = []
for tweet in tweetList_test:
    tweet_encoded = []
    for word in tweet.split():
        if hashEncoder.get(word) != None :
            word_encoded = hashEncoder[word]
            tweet_encoded.append(word_encoded)
    tweetList_test_encoded.append(tweet_encoded)

def decodeTweet(tweet_encoded):
    decodeList = []
    for word_encoded in tweetList_encoded[5]:
        decodeList.append(list(hashEncoder.keys())[list(hashEncoder.values()).index(word_encoded)])
    return decodeList
    
print('N = %d' % len(tweetList_encoded))
print('N_test = %d' % len(tweetList_test_encoded))
print()
print(tweetList[5])
print(tweetList_encoded[5])
#print(decodeTweet(tweetList_encoded[5]))
print()
print(tweetList_test[5])
print(tweetList_test_encoded[5])
print()

##############################################################

# find sentenses' max length
maxlen = 0
for tweet in tweetList_encoded:
    if len(tweet) > maxlen:
        maxlen = len(tweet)

##############################################################

N_train = (int)(0.9 * N)
N_valid = N - (int)(0.9 * N) - 1

tweetList_encoded_train = [tweetList_encoded[x] for x in range (N_train)]
tweetList_encoded_valid = [tweetList_encoded[x] for x in range (N_train + 1, N)]

labelList_train = [labelList[x] for x in range (N_train)]
labelList_valid = [labelList[x] for x in range (N_train + 1, N)]

print('N_train = %d' % N_train)
print('N_valid = %d' % N_valid)

N = 4743
N = 4743
N_test = 1701
M = 8787
N = 4743
N_test = 1701

when donald trump goes low   register to vote 
[2401, 7738, 5358, 8138, 122, 2994, 3026, 1289]

its nationalvoterregistrationday  celebrate by registering to vote → 
[694, 6083, 2711, 4934, 4669, 3026, 1289, 8604]

N_train = 4268
N_valid = 474


In [3]:
class SequenceData(object):
    
    def __init__(self, n_samples=1000, max_seq_len = 31, x_list=[], t_list=[]):
        
        self.data = []
        self.labels = []
        self.seqlen = []
        
        for i in range(n_samples):
            len1 = len(x_list[i])
            self.seqlen.append(len1)
            s = [x_list[i][j] for j in range(len1)]
            s += [0 for j in range(max_seq_len - len1)]
            self.data.append(s)
            
            if len(t_list) != 0:
                self.labels.append([t_list[i], 1 - t_list[i]])
            else:
                self.labels.append([1, 0])
            
        self.batch_id = 0

    def next(self, batch_size):

        if self.batch_id == len(self.data):
            self.batch_id = 0
            
        batch_data = (self.data[self.batch_id:min(self.batch_id + batch_size, len(self.data))])
        
        batch_labels = (self.labels[self.batch_id:min(self.batch_id + batch_size, len(self.data))])
        
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id + batch_size, len(self.data))])
        
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        
        return batch_data, batch_labels, batch_seqlen

In [4]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.01
training_steps = 10000
batch_size = 128
display_step = 1000

# Network Parameters
seq_max_len = maxlen # Sequence max length
word_features = 25
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
n_classes = 2 # linear sequence or notstate_size = 1#20

#trainset = SequenceData(n_samples = N_train,
#                           max_seq_len = seq_max_len,
#                           x_list = tweetList_encoded_train,
#                           t_list = labelList_train)

#testset = SequenceData(n_samples = N_valid,
#                          max_seq_len = seq_max_len,
#                          x_list = tweetList_encoded_valid,
#                          t_list = labelList_valid)

trainset = SequenceData(n_samples = N,
                           max_seq_len = seq_max_len,
                           x_list = tweetList_encoded,
                           t_list = labelList)

testset = SequenceData(n_samples = N_test,
                       max_seq_len = seq_max_len,
                       x_list = tweetList_test_encoded,
                       t_list = [])

# tf Graph input
x = tf.placeholder("int32", [None, seq_max_len])
y = tf.placeholder("float", [None, n_classes])

# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Embedding matrix
embeddings = tf.Variable(tf.random_uniform([M, word_features] , -1.0 , 1.0 ))

In [9]:
def neural_net_dense(x):
    layer_1 = tf.layers.dense(x, n_hidden_1,
                              kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001),
                              kernel_initializer = tf.truncated_normal_initializer(stddev=sqrt(2.0/n_hidden_1)))
    layer_2 = tf.layers.dense(layer_1, n_hidden_2,
                              kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001),
                              kernel_initializer = tf.truncated_normal_initializer(stddev=sqrt(2.0/n_hidden_2)))
    out_layer = tf.layers.dense(layer_2, n_classes,
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(0.001),
                                kernel_initializer = tf.truncated_normal_initializer(stddev=np.sqrt(2.0/n_classes)))
    return out_layer

In [10]:
def dynamicRNN(x, seqlen, embeddings):
    #
    x = tf.nn.embedding_lookup(embeddings, x)
    
    #
    x = tf.unstack(x, seq_max_len, 1)
    
    #
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(word_features)
    
    #
    outputs = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32, sequence_length=seqlen)[0]
    
    #
    outputs = tf.nn.dropout(outputs, keep_prob=0.6)


    #
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])
    
    #
    batch_size = tf.shape(outputs)[0]
    index = tf.range(batch_size) * seq_max_len + (seqlen - 1)
    outputs = tf.gather(tf.reshape(outputs, [-1, word_features]), index)
    
    #
    #return neural_net(outputs, weights, biases)
    return neural_net_dense(outputs)

In [11]:
pred = dynamicRNN(x, seqlen, embeddings)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
class_pred = tf.argmax(pred, axis=1)
prob_pred = tf.nn.softmax(pred)

correct_pred = tf.equal(tf.argmax(pred,axis=1), tf.argmax(y,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# saver
saver = tf.train.Saver(max_to_keep=1)

d:\anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [12]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    #saver.restore(sess, "./Model/model.ckpt")

    for step in range(1, training_steps+1):
        batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, 
                                       y: batch_y, 
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, 
                                                              y: batch_y, 
                                                              seqlen: batch_seqlen})
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")
    
    # Save Model
    saver.save(sess, "Model/model.ckpt")
    
    # Get Test Data
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen

    # Calculate accuracy
    #print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, 
    #                                                         y: test_label, 
    #                                                         seqlen: test_seqlen}))
    
    # Calculate loss
    #print("Testing Loss:", sess.run(cost, feed_dict={x: test_data, 
    #                                                 y: test_label, 
    #                                                 seqlen: test_seqlen}))
    
    # Calculate output
    test_result = sess.run(prob_pred, feed_dict={x: test_data, seqlen: test_seqlen})

Step 1, Minibatch Loss= 2.846292, Training Accuracy= 0.63281
Step 1000, Minibatch Loss= 0.459150, Training Accuracy= 0.78125
Step 2000, Minibatch Loss= 0.611614, Training Accuracy= 0.64844
Step 3000, Minibatch Loss= 0.053200, Training Accuracy= 1.00000
Step 4000, Minibatch Loss= 0.554319, Training Accuracy= 0.75781
Step 5000, Minibatch Loss= 0.427462, Training Accuracy= 0.82031
Step 6000, Minibatch Loss= 0.044267, Training Accuracy= 0.99219
Step 7000, Minibatch Loss= 0.339143, Training Accuracy= 0.85938
Step 8000, Minibatch Loss= 0.271917, Training Accuracy= 0.82812
Step 9000, Minibatch Loss= 0.070379, Training Accuracy= 0.99219
Step 10000, Minibatch Loss= 0.255168, Training Accuracy= 0.92188
Optimization Finished!


ValueError: setting an array element with a sequence.

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    saver.restore(sess, "Model/model.ckpt")
 
    # Get Test Data
    test_data = testset.data
    test_seqlen = testset.seqlen

    # Calculate output
    test_result = sess.run(prob_pred, feed_dict={x: test_data, seqlen: test_seqlen})

In [10]:
with open('test_result.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    fileheader = ["id", "realDonaldTrump", "HillaryClinton"]
    writer.writerow(fileheader)
    for i in range(len(test_result)):
        #onerow = [i, tweetList_test[i],test_result[i][1], test_result[i][0]]
        writer.writerow([i, test_result[i][1], test_result[i][0]])